Retropaper drive : https://drive.google.com/drive/folders/0B1GBCbARKSMzMnJ5dmx0TUVDblE

GIT : https://github.com/ThanitWin/SelfReferral/blob/master/AnalyzedData2ResultPresentation.ipynb

In [26]:
import numpy as np
import pandas as pd
from collections import Counter
from bokeh.plotting import figure, output_file, show
import sys
import datetime
import re
from matplotlib import pyplot as plt
import pprint
pp = pprint.PrettyPrinter(indent=4,depth=2)

# Define Function

In [5]:
global SAVETIC
global TIC_ITER
global TIC_TOC_LIST
TIC_TOC_LIST = []
SAVETIC= datetime.datetime.now()

def tic() :
    print("tic :: START, set iter = 0")
    global SAVETIC
    global TIC_ITER
    global TIC_TOC_LIST
    SAVETIC = datetime.datetime.now()
    TIC_ITER = 0
    TIC_TOC_LIST = []
    #return(SAVETIC)
def toc(print_every = 1) :
    global TIC_ITER
    global TIC_TOC_LIST
    TIC_ITER +=1
    TIC_TOC_LIST.append( str(datetime.datetime.now()-SAVETIC))
    if TIC_ITER%print_every ==0 :
        print("toc :: %04d"%TIC_ITER,datetime.datetime.now()-SAVETIC)

def gettictoc():
    return(TIC_TOC_LIST)

def dir2FinanceYM(year = 2013 ,month = 1):
    dirFinance = "OPD5258\\TT256888-FFO\\%d\\TT256888-FFO-%02d%d.csv"%(year,month,year);
    dirFinance = "Out_Patient_Data\\TT256888-FFO\\%d\\TT256888-FFO-%02d%d.csv"%(year,month,year);
    return(dirFinance )
def dir2InfoYM(year = 2013 ,month = 1):
    dirInfo = "Out_Patient_Data\\TT256888-ICD10\\%d\\%02d%d.csv"%(year,month,year);
    return(dirInfo)

# Change date --> day : Monday = 0
def wod(yyyymmdd):
    Y = int(str(yyyymmdd)[0:4])# - 543
    M = int(str(yyyymmdd)[4:6])
    D = int(str(yyyymmdd)[6:8])
    wd = datetime.date(Y,M,D).weekday()
    #days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    return(wd)


In [2]:
PathPertinent = "Pertinent_finding/"

# ==Presentation Goal==

## 0.1 Well Established Goal

1. First Episode
    1. Insurance_Group
        + Universal Coverage (UC)
        + Social Security Scheme (SSS)
        + Civil Servants’ Medical Benefit Scheme (CSMBS)
        + others
    1. Payment Coverage
        + Cover ~ DirectPay คือตัดผ่านห้องยาเลยไม่ต้องไปเบิกเอง จริงๆใช้ไม่ได้กับ CSMBS เพราะ CSMBS อาจไป
        + Not cover ~ 
    1. Demographic data over 7 years
    2. BOD subgroup : เลือกจาก http://bodthai.net/download/รายงานภาระโรค-2556/
        + I10 ไม่มีอย่างอื่นร่วม
        + E119 ไม่มีอย่างอื่นร่วม
        + J40 : Bronchitis
        + J43 : Emphysema
        + J450 : asthma
        + J44 Chronic COPD
        + A15 : TB Lung
        + A150-9 : TB Lung
        + Diarrhea หลากหลายมาก ยากเกินไป Peptic Ulcer --> แยกconditionที่ต้องส่องกล้องไมไ่ด้
    3. Trends of number, visit, cost, proportion, subgroup
    4. Statistical analysis : linear regressing for trend summaries 
2. Second Episode
    1. 


## 0.2 Persueing Goal

+ Option อื่นสำหรับ Primary Care ได้แก่
    + https://www.nuemd.com/icd-10/common-codes/primary-care
    + wiki : example: hypertension, angina, diabetes, asthma, COPD, depression and anxiety, back pain, arthritis or thyroid dysfunction.

In [ ]:
selectedOPD_Totally.csv
DemographicData_subgroupNoRight.csv
DemographicData_subgr_othUCWrong.csv
DemographicData_subgr_othSSWrong.csv
DemographicData_subgr_DisableWrong.csv
DemographicData_subgr_cost.csv
DemographicData.csv

# 1. Load Matched Data

In [6]:
path = "Intermediate/FilterMatchedSave/"
tic()
LoadLst = [0]*84
for itr in range(0,84):
    fileName = path+"allMatched%02d.csv"%itr
#     fileName = path+"filteredMatched%02d.csv"%itr
    LoadLst[itr] = pd.read_csv(fileName, encoding='utf-8-sig', sep=';')
    
    # เนื่องจากยังไม่ได้แปลง patient type ต้องแปลงก่อนใช้
    LoadLst[itr] = LoadLst[itr].rename(
    index = str, columns = {'patient_type_name_set': "right_at_payment_set", 
               'right_set' : "right_at_regist_set"}
    )
    toc(12)
temptb = LoadLst[0]

tic :: START, set iter = 0
toc :: 0012 0:00:22.662981
toc :: 0024 0:00:46.099196
toc :: 0036 0:01:09.579239
toc :: 0048 0:01:35.125612
toc :: 0060 0:02:01.671376
toc :: 0072 0:02:28.869959
toc :: 0084 0:02:56.795861


# 2. Assign [Insurance] and [Payment] Group

## 2.1 [run me] _is_?

In [7]:
#PaymentGroup Assignment : ต้อง run isGov, isUC, isSS
def isbotGov(tb):
    bool_srs= tb["right_at_regist_set"].str.contains('ธนาคารแห่งประเทศไทย')|\
            tb["right_at_payment_set"].str.contains('ธนาคารแห่งประเทศไทย')
    return(bool_srs)

def isbkkGov(tb):
    bool_srs= tb["right_at_regist_set"].str.contains('กทม')|\
            tb["right_at_payment_set"].str.contains('กทม')
    return(bool_srs)

def isgenGov(tb):
    bool_srs= tb["right_at_regist_set"].str.contains('ข้าราชการ')|\
            tb["right_at_payment_set"].str.contains('ข้าราชการ')
    return(bool_srs)

def isSS(tb):
    bool_srs= tb["right_at_regist_set"].str.contains('ประกันสังคม')|\
            tb["right_at_payment_set"].str.contains('ประกันสังคม')
    return(bool_srs)

def isUC(tb):
    bool_srs= tb["right_at_regist_set"].str.contains('บัตรทอง')|\
            tb["right_at_payment_set"].str.contains('บัตรทอง')
    return(bool_srs)

## 2.2 [run me] Add type

In [8]:

def addtype_n_DrtPay(tb): # function หลักเลย , Pass by reference
#     input : table ที่มี col [right_at_regist_set],[right_at_payment_set]
#     output : 

    #How to use: ใช้ update  ไม่มี return
    #    Update table ให้มี 
    #        'typePt' = type of patient  
    #        'drtPay' = Direct payment  --> เป็น boolean  ว่าได้เบิกจ่ายตรงจาก UC SS Gov หรือเปล่า สิทธิอื่นไม่สน จ่ายเพิ่มไม่เป็นไร 
    
    # UC,SS มีคำว่า ประกัน กับ บัตรทองเหมือนกันเลยกรองมาทีเดียวได้
    
    if False:#แก้ปัญหา Warning , Toggle off ได้ถ้าเกิดว่ามันทำให้ผลเพี้ยน
        tb = tb.reset_index(drop='index')
        # Problem is that it makes function becomes [Pass by value] rather than reference
    
    isbotGovsrs = isbotGov(tb)
    isbkkGovsrs = isbkkGov(tb)
    isgenGovsrs = isgenGov(tb)
    isSSsrs = isSS(tb)
    isUCsrs = isUC(tb)
    
    
    tb['typePt'] = isbotGovsrs|isbkkGovsrs|isgenGovsrs|isSSsrs|isUCsrs
    hasRight = isbotGovsrs|isbkkGovsrs|isgenGovsrs|isSSsrs|isUCsrs
    
    # สร้าง DirectPay ให้เป็นเหมือนกับ typePt เลย
    
    #====================== Type of Patient===============================
    
    #Mask Gov
        # ทำไปข้างบนแล้วเพราะ มี boolean แยกต่างหากสำหรับ Gov: bot,bkk,gen 
    
    # Mask SS
    SS_siMask = tb["right_at_regist_set"].str.contains('ผู้ป่วยประกันสังคมของ ร.พ.ศิริราช')|\
            tb["right_at_payment_set"].str.contains('ผู้ป่วยประกันสังคมของ ร.พ.ศิริราช')
    SS_othMask = tb["right_at_regist_set"].str.contains('ผู้ป่วยประกันสังคมของ ร.พ.อื่น')|\
            tb["right_at_payment_set"].str.contains('ผู้ป่วยประกันสังคมของ ร.พ.อื่น')|\
            tb["right_at_regist_set"].str.contains('สิทธิประกันสังคมกรณีทุพลภาพ')
        
    #Mask UC
    UC_siMask = tb["right_at_regist_set"].str.contains('บัตรทอง โรงพยาบาลศิริราช')|\
            tb["right_at_payment_set"].str.contains('บัตรทอง โรงพยาบาลศิริราช')
    UC_othMask = tb["right_at_regist_set"].str.contains('บัตรทอง โรงพยาบาลอื่น')|\
            tb["right_at_payment_set"].str.contains('บัตรทอง โรงพยาบาลอื่น')
    
    
    # Note : ธนาคารแห่งประเทศไทยเป็นfirst Priorityเลยเพราะว่า สิทธินี้ดีสุด และมีข้าราชการบางคนใช้สิทธิธนาคารแห่งประเทศไทย 
    # Heirachical Masking
        # Mask ธนาคารแห่งประเทศไทยหลังสุดจะได้ไม่โดนทับ --> กทม --> ราชการทั่วไป --> Mask SS --> Mask UC gเตรียมโดนทับได้เลย
        # mask ด้วย priority ต่ำกว่าก่อน แล้วค่อยทับด้วย priority สูงกว่าเรื่อยๆ
    tb['typePt'] = tb['typePt'].mask( UC_othMask,'othUC')
    tb['typePt'] = tb['typePt'].mask( UC_siMask ,'siUC')
    tb['typePt'] = tb['typePt'].mask( SS_othMask,'othSS')
    tb['typePt'] = tb['typePt'].mask( SS_siMask ,'siSS')
    tb['typePt'] = tb['typePt'].mask(isgenGovsrs,'genGov')
    tb['typePt'] = tb['typePt'].mask(isbkkGovsrs,'bkkGov')
    tb['typePt'] = tb['typePt'].mask(isbotGovsrs,'botGov')
    tb['typePt'] = tb['typePt'].astype(str) 
    
    useRight =  (\
                    tb["right_at_payment_set"].str.contains('ต้นสังกัด')|\
                    tb["right_at_payment_set"].str.contains('ข้าราชการ')|\
                    tb["right_at_payment_set"].str.contains('ธนาคารแห่งประเทศไทย')|\
                    tb["right_at_payment_set"].str.contains('กทม')|\
                    tb["right_at_payment_set"].str.contains('ประกันสังคม')|\
                    tb["right_at_payment_set"].str.contains('บัตรทอง')\
                )
    
    tb['drtPay'] = hasRight.mask(useRight,'hasClaim')
    tb['drtPay'] = tb['drtPay'].mask(hasRight & (~useRight),'noClaim')
    tb['drtPay'] = tb['drtPay'].mask(~hasRight,'noRight')
    tb['drtPay'] = tb['drtPay'].astype(str)
    #print(tb.columns)
    
def addtypeUCDrtPay(tb):
    # (SI, other) x ( drtPay, noRec) direct payment, no record
    tb['typeUC'] = isUC(tb)
    UC_siMask = tb["right_at_regist_set"].str.contains('บัตรทอง โรงพยาบาลศิริราช')|\
            tb["right_at_payment_set"].str.contains('บัตรทอง โรงพยาบาลศิริราช')
    UC_othMask = tb["right_at_regist_set"].str.contains('บัตรทอง โรงพยาบาลอื่น')|\
            tb["right_at_payment_set"].str.contains('บัตรทอง โรงพยาบาลอื่น')
    tb['typeUC'] = tb['typeUC'].mask(UC_siMask,'siUC')
    tb['typeUC'] = tb['typeUC'].mask(UC_siMask,'othUC')
    return(bool_temp)


def whichPaymentGroup():
    # there is 4 registered group
        # 1. UCsi
        # 2. UCother
        # 3. SSsi
        # 4. SSother
        # 5. Gov
#     output = UC,SP,
    pass

def isSelfPay(RightAtPaymentSet):
#     เนื่องจาก right@paymentSet --> อาจมีหลายสิทธิอยู่ด้วยกัน 
#     #case_gov
#     right_at_regist == 'ข้าราชการ' 'ธนาคารแห่งประเทศไทย'  
#     right_at_payment == 'ผู้ป่วยทั่วไป'
    
#     #case_non_gov
#     right_at_regist == any 
#         # ส่วนลดบุคลลากรก็ถือว่า selfpay
#     right_at_payment == 'ผู้ป่วยทั่วไป'
    
#     ข้าราชการ/เบิกจ่ายตรง กทม. --> ข้าราชการ เบิกจ่ายตรง กทม.: 0
#     ธนาคารแห่งประเทศไทย /  ทั่วไป --> 
#     ข้าราชการ --> ทั่วไป    : -1 ให้ 
#     Anything --> ทั่วไป    : 1
        
#     interpretation = 1'selfpay'
#     interpretation = 0'noselfpay'
#     interpretation = -1 'unsure' e.g. case government
    return(interpretation)


In [28]:
addtype_n_DrtPay(temptb)
pp.pprint(Counter(temptb.typePt)) 
pp.pprint(Counter(temptb.drtPay)) 

Counter({   'False': 68423,
            'genGov': 59003,
            'othUC': 20252,
            'siSS': 8750,
            'bkkGov': 4763,
            'siUC': 3955,
            'othSS': 2560,
            'botGov': 1168})
Counter({'hasClaim': 91183, 'noRight': 68423, 'noClaim': 9268})


## 2.3 [run me] ClinName2Code & dictionary 

<font color='red'>ตอนนี้แพทย์เวรยังไม่ exclude เวลาราชการ</font>

+ numCode2dpt_dict
+ numCode2exclude_dict
+ numCode2exclude4Gov_dict
+ numCode2surveyOPD_dict

In [29]:
def clinName2Code(word):
    #Extract [numCode] from [ThaiClinicName]
    # numCode is inthe form of (abcd) where abce is integer
    numlst = re.findall(r"\((\d+)\)",word)
    numlst = list(map(str,numlst))
    if len(numlst) > 0 :
        out = numlst[0]
    else :
        out = ''
        #out = re.search(r"\((\d+)\)",word).group(1)
    return(out)

# The Excels act as dictionary 
#     Name of Clinic
#     exclude
#     exclude4Gov
#     GIS
#     selectedOPD
#     investigation
#     lv_OPD

ClinicOverallTable = pd.read_excel("dict/clinicName2excl2select2dpt_20170426.xlsx",sheetname="dict")

ClinicOverallTable['numcode'] = ClinicOverallTable.NameOfClinic.map(clinName2Code)

# We have to clear all Non-code clinic out
ClinicOverallTable = ClinicOverallTable[ClinicOverallTable['numcode'] != '']

ClinicOverallTable.reindex()
numCode2dpt_dict = ClinicOverallTable.set_index('numcode')[['assignDepartment']].fillna('???').to_dict()
numCode2dpt_dict = numCode2dpt_dict['assignDepartment']
    
ClinicOverallTable.reindex()
numCode2exclude_dict = ClinicOverallTable.set_index('numcode')[['exclude']].fillna(0).to_dict()
numCode2exclude_dict = numCode2exclude_dict['exclude']

ClinicOverallTable.reindex()
numCode2exclude4Gov_dict = ClinicOverallTable.set_index('numcode')[['exclude4Gov']].fillna(0).to_dict()
numCode2exclude4Gov_dict = numCode2exclude4Gov_dict['exclude4Gov']

ClinicOverallTable.reindex()
numCode2surveyOPD_dict = ClinicOverallTable.set_index('numcode')[['selectedOPD']].fillna(0).to_dict()
numCode2surveyOPD_dict = numCode2surveyOPD_dict['selectedOPD']

In [35]:
numCode2exclude_dict

{'001': 1.0,
 '006': 1.0,
 '007': 1.0,
 '01': 0.0,
 '0101': 0.0,
 '0102': 0.0,
 '0103': 0.0,
 '0104': 0.0,
 '0105': 0.0,
 '0106': 0.0,
 '0107': 0.0,
 '0108': 0.0,
 '0109': 0.0,
 '0110': 0.0,
 '0111': 0.0,
 '0113': 0.0,
 '0114': 0.0,
 '0115': 555.0,
 '0116': 0.0,
 '0117': 0.0,
 '0118': 0.0,
 '0119': 0.0,
 '0120': 0.0,
 '0121': 0.0,
 '0122': 0.0,
 '0132': 0.0,
 '0136': 0.0,
 '0138': 0.0,
 '0147': 0.0,
 '0148': 0.0,
 '0149': 1.0,
 '015': 1.0,
 '0151': 0.0,
 '0152': 0.0,
 '0154': 0.0,
 '0155': 0.0,
 '0156': 1.0,
 '0157': 0.0,
 '0158': 0.0,
 '0160': 0.0,
 '0161': 1.0,
 '0162': 0.0,
 '0163': 0.0,
 '0165': 0.0,
 '0166': 0.0,
 '0167': 0.0,
 '0168': 0.0,
 '0169': 0.0,
 '0171': 0.0,
 '0172': 0.0,
 '0173': 0.0,
 '0174': 0.0,
 '0175': 0.0,
 '0181': 0.0,
 '0182': 0.0,
 '0183': 1.0,
 '0184': 0.0,
 '0185': 0.0,
 '0187': 1.0,
 '0188': 0.0,
 '0189': 0.0,
 '0190': 0.0,
 '0191': 1.0,
 '0192': 0.0,
 '0193': 0.0,
 '0194': 0.0,
 '0195': 0.0,
 '0196': 0.0,
 '0197': 0.0,
 '0198': 0.0,
 '0199': 1.0,
 '02': 0.0

## 2.4 [run me] Candidate Clinic ?

In [33]:
def clinSet2clinCodeSet(word):# this function is ??? used in Selected OPD
    # require : numCode2dpt_dict
    # input example : 'กิจกรรมบำบัด(1011)ตึกศรีสังวาลย์ ชั้น4||บำบัดทางกระดูกและข้อผู้ใหญ่(1009)ตึกศรีสังวาลย์ ชั้น3'
    # output example : 1011||1009
    numlst = re.findall(r"\((\d+)\)",word)
    numlst = list(map(str,numlst))
    joinedOutput = "||".join(numlst)
    return(joinedOutput)

def clinSet2dptSet(word): # this function is not used anymore
    # require : numCode2dpt_dict
    # input example : 'กิจกรรมบำบัด(1011)ตึกศรีสังวาลย์ ชั้น4||บำบัดทางกระดูกและข้อผู้ใหญ่(1009)ตึกศรีสังวาลย์ ชั้น3'
    # output example : rehab||rehab
    numlst = re.findall(r"\((\d+)\)",word)
    clincodelst = [ numCode2dpt_dict[numcode] for numcode in numlst]
    joinedOutput = "||".join(clincodelst)
    return(joinedOutput)

    #Example
    # temptb['dpt'] = temptb.clinic_set.map(clinSet2dptSet)
    # temptb.head(5)[['dpt','clinic_set']]

def codeSet2includeSet(word): # this function is not used anymore
    # require : numCode2exclude_dict
    # input example : 'กิจกรรมบำบัด(1011)ตึกศรีสังวาลย์ ชั้น4||บำบัดทางกระดูกและข้อผู้ใหญ่(1009)ตึกศรีสังวาลย์ ชั้น3'
    # output example : 1 || 1  (~ include || include)
    numlst = re.findall(r"\((\d+)\)",word)
    includeLst = [ int(numCode2exclude_dict[numcode]==0) for numcode in numlst]
    includeLst = map(str,includeLst)
    joinedOutput = "||".join(includeLst)
    return(joinedOutput)
    #Example
    # temptb['isExcludeSet'] = temptb.clinic_set.map(codeSet2includeSet)
    # temptb.head(5)[['isExcludeSet','clinic_set']]

def clinSet2included_dptSet(word): #USE!
    # require : numCode2dpt_dict
    # input example : 'ห้องตรวจอุบัติเหตุ (18)||บำบัดทางกระดูกและข้อผู้ใหญ่(1009)ตึกศรีสังวาลย์ ชั้น3'
    # intermediate example : surgery || rehab
    # output example :  rehab
    numlst = re.findall(r"\((\d+)\)",word)
    includedDptLst = [ numCode2dpt_dict[numcode] for numcode in numlst if numCode2exclude_dict[numcode]==0 ]
    joinedOutput = "||".join(includedDptLst)
    
    return(joinedOutput)

def clinSet2included_dpt4GovSet(word): #USE! for Goverment Record Selection
    # require : numCode2dpt_dict
    # input example : 'ห้องตรวจอุบัติเหตุ (18)||บำบัดทางกระดูกและข้อผู้ใหญ่(1009)ตึกศรีสังวาลย์ ชั้น3'
    # intermediate example : surgery || rehab
    # output example :  rehab
    numlst = re.findall(r"\((\d+)\)",word)
    includedDptLst = [ numCode2dpt_dict[numcode] for numcode in numlst if numCode2exclude4Gov_dict[numcode]==0 ]
    joinedOutput = "||".join(includedDptLst)    
    return(joinedOutput)

temptb['dpt_set'] = temptb.clinic_set.map(clinSet2included_dptSet)
temptb['candidate'] = (temptb['dpt_set']!='')
addtype_n_DrtPay(temptb)
temptb.head(5)[['dpt_set','candidate','clinic_set','typePt','drtPay']]


,dpt_set,candidate,clinic_set,typePt,drtPay
0,rehab||rehab,True,กิจกรรมบำบัด(1011)ตึกศรีสังวาลย์ ชั้น4 ...,False,noRight
1,rehab||rehab,True,กิจกรรมบำบัด(1011)ตึกศรีสังวาลย์ ชั้น4 ...,False,noRight
2,,False,ห้องตรวจอุบัติเหตุ (18) ...,False,noRight
3,,False,ห้องตรวจอุบัติเหตุ (18) ...,False,noRight
4,med,True,คลีนิก lab(90)ตึกผู้ป่วยนอก ชั้น 1 ...,othUC,hasClaim


#### CountCoverageNClaim_appropriate_hour :

อันนี้เป็นแม่แบบให้กับ Demographic ด้วย
คือ count coverage Claim โดยที่ gov นับรวม คลินิกนอกเวลาด้วย

ตอนนี้งงอยู่ว่าเวลาหายไปไหน หรือว่าเรา exclude

In [ ]:
ListOfDct = []
tic()
for temptb in LoadLst[80:84]:
    temptb['dpt_set'] = temptb.clinic_set.map(clinSet2included_dptSet)
    temptb['dpt4Gov_set'] = temptb.clinic_set.map(clinSet2included_dpt4GovSet)
    addtype_n_DrtPay(temptb)# add 'typePt','drtPay'
    isGov_srs = temptb['typePt'].str.contains('Gov')
    temptb['appropriate_dpt_set'] = temptb['dpt_set'].copy()
    if True : # toggle True ถ้าอยากให้ ราชการ รวม คลินิกนอกเวลาด้วย
        temptb['appropriate_dpt_set'][isGov_srs] = temptb['dpt4Gov_set'][isGov_srs]
    hasAtLeast1Dpt_tb = temptb[ (temptb['appropriate_dpt_set']!='')]
    
    #addtype_n_DrtPay(hasAtLeast1Dpt_tb)
    toc(12)
#     [['candidate','clinic_set','typePt','drtPay']]
    tmp =  hasAtLeast1Dpt_tb\
        .sort_values(['typePt','drtPay'])\
        .groupby(['typePt','drtPay']).count()['sex']

    ListOfDct.append(tmp.to_dict())


# Export to CSV&Excel
tempFileName = "Pertinent_finding/CountCoverageNClaim_appropriate_hour__"
# Save : as-csv
pd.concat(map(pd.Series,ListOfDct), axis=1).reset_index()\
.fillna(0)\
.to_csv(tempFileName+'.csv', encoding='utf-8-sig')

# Save as Excel
writer = pd.ExcelWriter(tempFileName+'.xls')
pd.concat(map(pd.Series,ListOfDct), axis=1).reset_index()\
.fillna(0)\
.to_excel(writer,'Sheet1')
writer.save()

# END
